In [66]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV

In [32]:
# 데이터 불러오기
wine = pd.read_csv("./data/08_wine.csv")

# 훈련데이터(input, target) 생성
wine_input = wine[["alcohol","sugar","pH"]].to_numpy() # 2차원
wine_target = wine["class"].to_numpy() # 1차원

# 훈련데이터와 테스트데이터로 분리
train_input, test_input, train_target, test_target = train_test_split(wine_input,wine_target,
                                                                      test_size=0.2,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [33]:
# 일반적인 데이터 분류 기준
# 훈련 : 검증 : 테스트 = 6 : 2 : 2
# 훈련데이터를 sub와 val로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(train_input,train_target,
                                                                      test_size=0.2,random_state=42)
print(sub_input.shape,sub_target.shape)
print(val_input.shape,val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [34]:
# DT 훈련
# 모델생성
dt = DecisionTreeClassifier(random_state=42)
# 훈련
dt.fit(sub_input,sub_target)
# 검증
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [35]:
# 테스트데이터의 정확도
print(dt.score(test_input,test_target))

0.8569230769230769


In [58]:
# 위 과정을 한번에 묶어놓은 교차검증 알고리즘
# 교차검증
"""
- 내부적으로 훈련/검증 데이터로 구분하여 사용하는 알고리즘
- 내주적으로 구분에 사용하는 데이터는 위에서 구분한 train 데이터 사용
- 데이터 준비는 기존과 동일하게 훈련 및 테스트 데이터만 있으면 됨
"""
# train_input을 다섯등분 -> 마지막을 타겟 -> 나머지 4개로 훈련시킴 -> 타겟을 옮기고, 나머지 4개로 훈련 -> 
# 총 5번 훈련 -> 폴드 방식
# 교차검증 주요 키워드 : 폴드
"""
- 훈련데이터(train_input)를 폴드의 개수만큼 나누어서 교차하면서 훈련
- 각 폴더 단위별 훈련한 검증 결과를 평균한 값이 최종 검증 결과가 됨
"""

# 교차검증 수행
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00699902, 0.00507069, 0.00403214, 0.00402594, 0.0049088 ]), 'score_time': array([0.00100064, 0.0009644 , 0.00100255, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [59]:
# 폴드 값 5개 평균내기
np.mean(scores["test_score"])

0.855300214703487

In [62]:
### 폴드 분할 컨트롤 하기

"""
- cv에 StratifiedFold()
- 클래스에 아무값도 넣지 않으면 기본 폴드는 5를 사용함


"""

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores["test_score"]))

0.855300214703487


In [65]:
# n_splits : 훈련데이터를 몇 개 폴드로 나눌것인가
# shuffle : 훈련데이터 섞기(기본값 : False)
splitter = StratifiedKFold(n_splits=10,
                           shuffle=True,
                           random_state=42)

scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores["test_score"]))

0.8574181117533719


In [67]:
# 하이퍼파라미터 튜닝(AutoML)
"""
- 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행하는 개념
- 사용패키지 : sklearn.model_selection
- 사용클래스 : GridSearchCV
"""
# 트리의 깊이(max_depth)값 찾기
# max_dedth의 값을 찾기 위한 범위 지정
params = {"max_depth" : range(5,20,1)}
# 하이퍼파라미터 그리드 서치 객체 생성
# n_jobs : 사용할 CPU코어 개수 지정(-1은 모든 코어 사용)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  params,n_jobs=-1)
# 스스로 하이퍼파라미터 찾기
gs.fit(train_input,train_target)
# 자동으로 찾은 하이퍼파라미터 값 출력
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [69]:
# 검증
dt = gs.best_estimator_
print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))

0.9003271117952665
0.8584615384615385
